In [1]:
!pip install alpaca_trade_api

     |████████████████████████████████| 81kB 4.3MB/s 
     |████████████████████████████████| 204kB 24.6MB/s 


In [2]:
!pip install import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=9b64be4c7e8b12a0b1f166a03bfc6aa6300efadb9059f4170a2670f8e3bb9974
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [3]:
!pip install ta

  Created wheel for ta: filename=ta-0.7.0-cp36-none-any.whl size=28718 sha256=daa2b0380da2caf0cdf4b6b699b7359b14109b316af544e5edc6efe352d9ecd1
  Stored in directory: /root/.cache/pip/wheels/dd/88/30/de9553fb54a474eb7480b937cdbb140bdda613d29cf4da7994
Successfully built ta


In [4]:
!pip install bta-lib

     |████████████████████████████████| 102kB 4.2MB/s 


In [6]:
from google.colab import drive
import sys

In [9]:
drive.mount('/content/drive') # mount google drive folder

Mounted at /content/drive


In [10]:
sys.path.append('/content/drive/') # append path

In [12]:
%cd /content/drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [13]:
# imports
# if any are unable to run, make a new cell and use !pip install <package name>
from urllib.error import HTTPError
import import_ipynb
import btalib, requests, json, time, ta, time, math
import config
import pandas as pd
from datetime import datetime, timedelta, date
import warnings
from pandas.core.common import SettingWithCopyWarning
import alpaca_trade_api as tradeapi

importing Jupyter notebook from config.ipynb


In [28]:
# SCRAPE SCRIPT

def scrape():
  global buy_stocks
  ##################################################-SETUP-##################################################
  api = tradeapi.REST(config.APCA_API_KEY_ID, config.APCA_API_SECRET_KEY, config.APCA_API_BASE_URL) # set URLs
  portfolio = api.list_positions()


  ##################################################-GET LIST OF SYMBOLS-##################################################
  holdings=open('data/holdings.csv').readlines() # read csv of QQQ holdings - should be able to use any csv
  #holdings=open('data/WILSHIRE-5000-Stock-Tickers-List.csv').readlines() # read csv of Wilshire 5000 - not using right now as I can only pass in 200 symbols

  # pull symbols from holdings csv, assigns it to symbols variable (list of symbols)
  symbols = [holding.split(',')[2].strip() for holding in holdings][1:] # use this line for QQQ holdings
  #symbols = [holding.split(',')[0].strip() for holding in holdings][1:] # use this line for Wilshire 5000
  symbols = ",".join(symbols)

  day_bars_url = '{}/day?symbols={}&limit=201'.format(config.BARS_URL, symbols) #get daily data every day for 200 days (for each symbol)
  r=requests.get(day_bars_url, headers=config.HEADERS) # use requests module to search each URL
  data = r.json() # gives us the data in a dictionary


  ##################################################-SET UP DATA CONTAINERS-##################################################
  df = pd.DataFrame() # create empty dataframe
  # create empty lists for each piece of data we're scraping
  time_list = []
  open_list = []
  high_list = []
  low_list = []
  close_list = []
  volume_list = []
  symbol_list = []


  ##################################################-SCRAPE DATA-##################################################
  for symbol in data:
      for bar in data[symbol]:
          t = datetime.fromtimestamp(bar['t']) # change from UNIX timestamp to datetime object
          day = t.strftime('%Y-%m-%d') # save to day variable
          # append each variable to its own list, will add to df later
          time_list.append(day)
          open_list.append(bar['o'])
          high_list.append(bar['h'])
          low_list.append(bar['l'])
          close_list.append(bar['c'])
          volume_list.append(bar['v'])
          symbol_list.append(symbol)
  # append each list to its own column in df
  df['symbol'] = symbol_list
  df['time'] = time_list
  df['open'] = open_list
  df['high'] = high_list
  df['low'] = low_list
  df['close'] = close_list
  df['volume'] = volume_list


  ##################################################-ADDING IN INDICATORS-##################################################
  sma2 = btalib.sma(df, period=2)
  df['sma2']=sma2.df
  sma5 = btalib.sma(df, period=5)
  df['sma5']=sma5.df
  sma10 = btalib.sma(df, period=10)
  df['sma10']=sma10.df
  sma20 = btalib.sma(df, period=20)
  df['sma20']=sma20.df
  sma200 = btalib.sma(df, period=200)
  df['sma200']=sma200.df
  df['rsi'] = ta.momentum.rsi(df.close, window=6) # btalib rsi not working, we will use talib for this indicator


  ##################################################-CALCULATE PIVOT POINT AND RESISTANCE LEVEL-##################################################
  df['pivot_point'] = (df['high'].shift(1) + df['low'].shift(1) + df['close'].shift(1))/3
  df['r1'] = (2*df['pivot_point']) - df['low'].shift(1)
  df['s1'] = (2*df['pivot_point']) - df['high'].shift(1)
  df['r2'] = (df['pivot_point'] - df['s1']) + (df['r1'])
  df['take_profit'] = ((df['r2'] - df['close'].shift(1))*.75) + df['close'].shift(1)

  # for now, we will just trade on these levels
  # if more are needed, they can be found and explained here:
  # https://www.daytrading.com/pivot-points#:~:text=Calculation%20of%20Pivot%20Points,-Pivots%20points%20can&text=The%20central%20price%20level%20%E2%80%93%20the,or%20period%2C%20more%20generally).&text=Resistance%201%20%3D%20(2%20x%20Pivot,)%20%E2%80%93%20High%20(previous%20period)

  ##################################################-ADDING IN TIME-##################################################
  # add in time - make sure the bot is not trying to trade based on the price of non-trading days
  today = date.today()

  # conditions to use last trading days as variables
  if today.weekday() == 6: # sunday
      yesterday = today - timedelta(days=2)
      yesterday = yesterday.strftime('%Y-%m-%d')

      two_days_ago = today - timedelta(days=3)
      two_days_ago = two_days_ago.strftime('%Y-%m-%d')

      three_days_ago = today - timedelta(days=4)
      three_days_ago = three_days_ago.strftime('%Y-%m-%d')

      four_days_ago = today - timedelta(days=5)
      four_days_ago = four_days_ago.strftime('%Y-%m-%d')

  if today.weekday() == 0: # monday
      yesterday = today - timedelta(days=3)
      yesterday = yesterday.strftime('%Y-%m-%d')

      two_days_ago = today -timedelta(days=4)
      two_days_ago = two_days_ago.strftime('%Y-%m-%d')

      three_days_ago = today - timedelta(days=5)
      three_days_ago = three_days_ago.strftime('%Y-%m-%d')

      four_days_ago = today - timedelta(days=6)
      four_days_ago = four_days_ago.strftime('%Y-%m-%d')

  if today.weekday() == 1: # tuesday
      yesterday = today - timedelta(days=1)
      yesterday = yesterday.strftime('%Y-%m-%d')

      two_days_ago = today - timedelta(days=4)
      two_days_ago = two_days_ago.strftime('%Y-%m-%d')

      three_days_ago = today - timedelta(days=5)
      three_days_ago = three_days_ago.strftime('%Y-%m-%d')

      four_days_ago = today - timedelta(days=6)
      four_days_ago = four_days_ago.strftime('%Y-%m-%d')

  if today.weekday() == 2: # wednesday
      yesterday = today - timedelta(days=1)
      yesterday = yesterday.strftime('%Y-%m-%d')

      two_days_ago = today - timedelta(days=2)
      two_days_ago = two_days_ago.strftime('%Y-%m-%d')

      three_days_ago = today - timedelta(days=5)
      three_days_ago = three_days_ago.strftime('%Y-%m-%d')

      four_days_ago = today - timedelta(days=6)
      four_days_ago = four_days_ago.strftime('%Y-%m-%d')

  if today.weekday() == 3: # thursday
      yesterday = today - timedelta(days=1)
      yesterday = yesterday.strftime('%Y-%m-%d')

      two_days_ago = today - timedelta(days=2)
      two_days_ago = two_days_ago.strftime('%Y-%m-%d')

      three_days_ago = today - timedelta(days=3)
      three_days_ago = three_days_ago.strftime('%Y-%m-%d')

      four_days_ago = today - timedelta(days=6)
      four_days_ago = four_days_ago.strftime('%Y-%m-%d')

  if today.weekday() == 4 or today.weekday() == 5: # friday or saturday
      yesterday = today - timedelta(days=1)
      yesterday = yesterday.strftime('%Y-%m-%d')

      two_days_ago = today - timedelta(days=2)
      two_days_ago = two_days_ago.strftime('%Y-%m-%d')

      three_days_ago = today - timedelta(days=3)
      three_days_ago = three_days_ago.strftime('%Y-%m-%d')

      four_days_ago = today - timedelta(days=4)
      four_days_ago = four_days_ago.strftime('%Y-%m-%d')

  today = today.strftime('%Y-%m-%d')


  ##################################################-FILTER DATA-##################################################
  # first, let's only work with the last three days of data
  big_money_df = df.loc[(df['time']==today) |
                        (df['time']==yesterday) |
                        (df['time']==two_days_ago) |
                        (df['time']==three_days_ago)] # only take records from up to three days ago


  warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
  # warning being ignored - despite the error, I am still getting the expected result
  big_money_df.loc[:, 'above_sma10'] = (big_money_df['close'] > big_money_df['sma10']) # a simple column to detect if a stock is above or below the SMA 10-day line
  # setwithcopy warning - uncomment the following to ensure the code worked correctly
  # print((big_money_df['close'] > big_money_df['sma5']).sum())
  # print(big_money_df['above_sma5'].value_counts())
  # first value should match the true values
  buy_stocks = big_money_df.loc[

                              ##########-RSI LESS THAN 70-##########
                              (big_money_df['rsi'] < 70)


                              ##########-CHECK IF SMA LINE HAS BEEN CROSSED-##########
                              # three days ago, the price action was below the SMA line
                              # two days ago and yesterday, the price action was above the SMA line
                              # we want to know if the price action has held above the SMA line for two days in a row after being below it
                              & (big_money_df['above_sma10'].shift(3) == False) # below SMA 3 days ago
                              & (big_money_df['above_sma10'].shift(2) == True) # above SMA 2 days ago
                              & (big_money_df['above_sma10'].shift(1) == True) # above SMA yesterday
                              & (big_money_df['above_sma10'] == True) # above SMA today
                              & (big_money_df['close'].shift(1) > big_money_df['sma200'])


                              ##########-CHECK IF SHORT TERM SMA > LONG TERM SMA FOR THREE DAYS-##########
                              & (big_money_df['sma10'] > big_money_df['sma200']) # today
                              & (big_money_df['sma10'].shift(1) > big_money_df['sma200'].shift(1)) # yesterday
                              & (big_money_df['sma10'].shift(2) > big_money_df['sma200'].shift(2)) # two days ago


                              ##########-ONLY RETURN RECORDS FROM TODAY-##########
                              & (big_money_df['time']==today)

                              ##########-CLOSE IS LESS THAN RESISTANCE LEVEL 2-##########
                              & (big_money_df['close'].shift(1) <=big_money_df['r2'])

                              & (big_money_df['close'].shift(1) <= big_money_df['take_profit']) # CURRENT PRICE REACHES 75% OF RESISTANCE LEVEL 2 - NOT CURRENTLY USING


                                  ]


  buy_stocks = buy_stocks.sort_values(by=['rsi']) # sort values by RSI

  # don't buy more of a stock that we already own our desired equity of
  my_symbols = []
  for symbol in portfolio:
      my_symbols.append(symbol.symbol)

  buy_stocks = buy_stocks[~buy_stocks.symbol.isin(my_symbols)]

  #print(buy_stocks[['symbol', 'close', 'sma10', 'above_sma10', 'rsi']])
  # print('these are the best stocks to buy, if available:')
  #print(buy_stocks[['symbol', 'close', 'sma10', 'sma200', 'r2', 'rsi']])
  #print(buy_stocks[['symbol', 'close', 'take_profit', 'r2']])
buy_stocks = buy_stocks

In [30]:
# BUY SCRIPT

def buy(buy_stocks=buy_stocks):

  ##################################################-SETUP-##################################################
  api = tradeapi.REST(config.APCA_API_KEY_ID, config.APCA_API_SECRET_KEY, config.APCA_API_BASE_URL) # set URLs
  account = api.get_account() # get account info
  print('${} is available as buying power.'.format(account.buying_power)) # check buying power


  ##################################################-FINAL CRITERIA FOR BUYING-##################################################
  print('these are the best stocks to buy, if available:')
  print(buy_stocks[['symbol', 'close', 'sma10', 'sma200', 'rsi']])
  buy_stocks = buy_stocks['symbol'].tolist() # create a list of the stocks above
  buy_stocks_list = [] # final buy list

  for stock in buy_stocks:
      # we want to ensure we can afford each stock on our buy list
      # this loop filters out stocks we cannot afford by taking each element in buy_stocks, pulling its current price from the API, and adding it to our new list
      barset = api.get_barset(stock, '1Min', limit=1)
      stock_price = barset[stock][0].c #get current price of stock
      if stock_price < float(account.buying_power): # check if the stock's price is less than our buying power
          buy_stocks_list.append(stock)

  # might add current price scraper here if needed


  ##################################################-BUY STOCKS-##################################################
  while True: # will break when I don't want the bot to buy more stocks
      account = api.get_account() # refresh account info
      portfolio = api.list_positions()
      if buy_stocks_list: # check if there are stocks to buy

          for stock in buy_stocks_list:
              """
              This loop ensures that we buy a similar ratio of each stock rather than buying the same quantity of each stock.
              If one stock has a price of $1000 and one has a price of $100, for example, we don't want ten shares of each stock.
              We'd rather have one share of stock one and ten shares of stock two to ensure our portfolio is more diversified
              """
              barset = api.get_barset(stock, '1Min', limit=1)
              stock_price = barset[stock][0].c #get current price
              equity_limit = 600 # maximum equity you want to own of each stock
              buy_qty = 0

              while equity_limit > stock_price:
                  buy_qty += 1 # increase buy quantity
                  equity_limit -= stock_price # decrease equity_limit variable by the stock price after each iteration

              # the following is currently not being used, but I will leave it here in case I want to use it again
              # if equity_limit < float(account.buying_power):
              #     buy_qty = math.floor(float(account.buying_power)/stock_price) # buy maximum number of stocks available with our buying power
              # else:
              #     pass

              try:
                  # submit order for each stock in loop w/ our qty determined by our ratio calculator above
                  api.submit_order(
                      symbol=stock,
                      qty=buy_qty,
                      side='buy',
                      type='market',
                      time_in_force='gtc'
                  )
                  print(f'{buy_qty} shares of {stock} will be bought')
              except (tradeapi.rest.APIError):
                  print("Insufficient buying power for best available stocks")
                  break

      else:
          print("Either none of our scanned stocks meet our buying criteria or you don't have sufficient buying power")
          break
      break


In [31]:
# SELL SCRIPT

def sell():
  ##################################################-SETUP-##################################################
  api = tradeapi.REST(config.APCA_API_KEY_ID, config.APCA_API_SECRET_KEY, config.APCA_API_BASE_URL)
  portfolio = api.list_positions() # get account info


  ##################################################-FIND CURRENT POSITIONS-##################################################
  # create empty list for current holdings and quantity held
  current_holdings_df = pd.DataFrame()
  current_holdings = []
  holding_qty = []
  equity_owned = []
  current_price = []

  # loop used for getting data about currently owned stocks
  i=0 # used for indexing each item in our portfolio - will increase with each iteration of the loop
  for stock in portfolio:
      current_holdings.append(portfolio[i].symbol)
      holding_qty.append(portfolio[i].qty)
      equity_owned.append(portfolio[i].market_value)
      current_price.append(portfolio[i].current_price)
      i+=1

  # append each list to our dataframe
  current_holdings_df['symbol'] = current_holdings

  holding_qty = [float(i) for i in holding_qty]
  current_holdings_df['qty_owned'] = holding_qty

  equity_owned = [float(i) for i in equity_owned]
  current_holdings_df['equity_owned'] = equity_owned

  current_price = [float(i) for i in current_price]
  current_holdings_df['current_price'] = current_price


  ##################################################-SCRAPE DATA FOR CURRENT HOLDINGS-##################################################
  symbols = current_holdings

  symbols = [symbol.split(',')[0].strip() for symbol in symbols] # use this line for QQQ holdings
  #symbols = [holding.split(',')[0].strip() for holding in holdings][1:] # use this line for Wilshire 5000
  symbols = ",".join(symbols)

  day_bars_url = '{}/day?symbols={}&limit=201'.format(config.BARS_URL, symbols) #get daily data every day for 200 days (for each symbol)
  r=requests.get(day_bars_url, headers=config.HEADERS) # use requests module to search each URL
  data = r.json() # gives us the data in a dictionary


  ##################################################-SET UP DATA CONTAINERS-##################################################
  df = pd.DataFrame() # create empty dataframe
  # create empty lists for each piece of data we're scraping
  time_list = []
  open_list = []
  high_list = []
  low_list = []
  close_list = []
  volume_list = []
  symbol_list = []


  ##################################################-SCRAPE DATA-##################################################
  for symbol in data:
      for bar in data[symbol]:
          t = datetime.fromtimestamp(bar['t']) # change from UNIX timestamp to datetime object
          day = t.strftime('%Y-%m-%d') # save to day variable
          # append each variable to its own list, will add to df later
          time_list.append(day)
          open_list.append(bar['o'])
          high_list.append(bar['h'])
          low_list.append(bar['l'])
          close_list.append(bar['c'])
          volume_list.append(bar['v'])
          symbol_list.append(symbol)
  # append each list to its own column in df
  df['symbol'] = symbol_list
  df['time'] = time_list
  df['open'] = open_list
  df['high'] = high_list
  df['low'] = low_list
  df['close'] = close_list
  df['volume'] = volume_list


  ##################################################-CALCULATE PIVOT POINT AND RESISTANCE LEVEL-##################################################
  df['pivot_point'] = (df['high'].shift(1) + df['low'].shift(1) + df['close'].shift(1))/3
  df['r1'] = (2*df['pivot_point']) - df['low'].shift(1)
  df['s1'] = (2*df['pivot_point']) - df['high'].shift(1)
  df['r2'] = (df['pivot_point'] - df['s1']) + (df['r1'])
  df['take_profit'] = ((df['r2'] - df['close'].shift(1))*.75) + df['close'].shift(1)


  ##################################################-ADDING IN INDICATORS-##################################################
  sma2 = btalib.sma(df, period=2)
  df['sma2']=sma2.df
  sma5 = btalib.sma(df, period=5)
  df['sma5']=sma5.df
  sma10 = btalib.sma(df, period=10)
  df['sma10']=sma10.df
  sma20 = btalib.sma(df, period=20)
  df['sma20']=sma20.df
  sma200 = btalib.sma(df, period=200)
  df['sma200']=sma200.df
  df['rsi'] = ta.momentum.rsi(df.close, window=6, fillna=False) # btalib rsi not working, we will use talib for this indicator


  ##################################################-JOIN THE TWO DATAFRAMES-##################################################
  today = str(date.today())
  df = df.loc[df['time']==today] # we only want today's records

  df = df.merge(current_holdings_df, on='symbol', how='inner')


  ##################################################-CRITERIA FOR SELLING-##################################################
  sell_df = df.loc[

                          ##########-(RSI > 70)-##########
                          (df['rsi'] > 70)

                          ##########-CLOSE HOLDS BELOW SMA10 LINE-##########
                      |   (
                          (df['close'].shift(1) < df['sma10'].shift(1)) & (df['close'] < df['sma10'])
                          )

                          ##########-CURRENT PRICE REACHES RESISTANCE LEVEL 2-##########
                      |   (df['current_price'] >= df['take_profit']) # CURRENT PRICE REACHES 75% OF RESISTANCE LEVEL 2 - NOT CURRENTLY USING
                  #    | (df['current_price'] >=df['r2'])


                          ##########-CLOSE DIPS BELOW LONG-TERM SMA-##########
                      |   (df['close'] < df['sma200'])

                      ]


  ##################################################-SELL STOCKS-##################################################
  if sell_df.empty == False: # if there are stocks to sell
      portfolio = api.list_positions()
      print(f"stocks being sold: {list(sell_df['symbol'])}")
      i=0 # used for indexing
      for stock in sell_df['symbol']:
          try:
              api.submit_order(
                  symbol=sell_df['symbol'].iloc[i],
                  qty=sell_df['qty_owned'].iloc[i],
                  side='sell',
                  type='market',
                  time_in_force='gtc')
              print(f'{stock} sold')
              i+=1
          except (requests.exceptions.HTTPError, tradeapi.rest.APIError):
              print(f"Either your order to sell {stock} hasn't been filled, or daytrade protection has been activated")
              continue


In [33]:
# Final cell - run this continuously to operate the bot

while True:
  scrape()
  buy()
  sell()
  #print('waiting two minutes')
  time.sleep(60)
  print("-"*50)
  continue

$274.89 is available as buying power.
these are the best stocks to buy, if available:
     symbol   close     sma10      sma200        rsi
8039   FAST   49.82   48.7020   42.315050  65.382329
4622   CHKP  123.12  120.7221  113.781917  67.015535
Insufficient buying power for best available stocks
stocks being sold: ['CMCSA', 'DXCM', 'EXPE', 'PCAR']
Either your order to sell CMCSA hasn't been filled, or daytrade protection has been activated
Either your order to sell DXCM hasn't been filled, or daytrade protection has been activated
Either your order to sell EXPE hasn't been filled, or daytrade protection has been activated
Either your order to sell PCAR hasn't been filled, or daytrade protection has been activated


KeyboardInterrupt: ignored